In [1]:
!pip install numpy pandas opencv-python tensorflow keras matplotlib


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   -------------------------- ------------- 0.8/1.2 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
import matplotlib.pyplot as plt
import os


In [32]:
def extract_frames(video_path, resize_dim=(256, 256)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to the desired dimensions
        frame = cv2.resize(frame, resize_dim)
        # Convert to grayscale (optional, depending on your diffusion model)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Example usage:
video_path = r"C:\Users\ngaga\Downloads\abuse 001 cropped.mp4"  # Replace with your video file path
frames = extract_frames(video_path)
print(f"Extracted {len(frames)} frames from the video.")


Extracted 766 frames from the video.


In [33]:
def build_diffusion_model(input_shape):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])
    return model

input_shape = (256, 256, 1)  # Assuming grayscale frames
diffusion_model = build_diffusion_model(input_shape)
diffusion_model.summary()


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_30 (Conv2D)                   │ (None, 256, 256, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 128, 128, 16)        │           4,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_32 (Conv2D)                   │ (None, 64, 64, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_10 (UpSampling2D)      │ (None, 128, 128, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_33 (Conv2D)                   │ (None, 128, 128, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_11 (UpSampling2D)      │ (None, 256, 256, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_34 (Conv2D)                   │ (None, 256, 256, 1)         │             289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,193 (47.63 KB)

 Trainable params: 12,193 (47.63 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
def get_reconstruction_errors(diffusion_model, frames):
    # Normalize frames to 0-1 range
    frames = frames / 255.0
    frames = frames[..., np.newaxis]  # Add channel dimension
    reconstructions = diffusion_model.predict(frames)
    
    # Calculate the reconstruction error for each frame
    reconstruction_errors = np.mean(np.square(frames - reconstructions), axis=(1, 2, 3))
    return reconstruction_errors

# Generate reconstruction errors
reconstruction_errors = get_reconstruction_errors(diffusion_model, frames)
print("Reconstruction errors calculated for each frame.")


24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 169ms/step
Reconstruction errors calculated for each frame.


In [35]:
def build_cnn_lstm_model(frame_shape, sequence_length=10):
    model = models.Sequential()
    # CNN part for feature extraction
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(sequence_length, *frame_shape)))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    # LSTM part for temporal analysis
    model.add(LSTM(64, activation='relu', return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

frame_shape = (256, 256, 1)
cnn_lstm_model = build_cnn_lstm_model(frame_shape)
cnn_lstm_model.summary()


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_15                  │ (None, 10, 254, 254, 32)    │             320 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_16                  │ (None, 10, 127, 127, 32)    │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_17                  │ (None, 10, 516128)          │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │     132,145,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 132,145,793 (504.10 MB)

 Trainable params: 132,145,793 (504.10 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Prepare the data for the CNN+LSTM
def prepare_sequences(frames, sequence_length=10):
    sequences = []
    for i in range(len(frames) - sequence_length + 1):
        sequence = frames[i:i + sequence_length]
        sequences.append(sequence)
    return np.array(sequences)

# Create sequences
sequence_length = 10
sequences = prepare_sequences(frames, sequence_length)

# Reshape sequences for CNN+LSTM
sequences = sequences / 255.0
sequences = sequences[..., np.newaxis]

# Example of running the model (using random weights for illustration)
predictions = cnn_lstm_model.predict(sequences)
print(f"Predictions: {predictions}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step
Predictions: [[0.52605164]
 [0.5259824 ]
 [0.52586865]
 [0.52581155]
 [0.52571726]
 [0.5256667 ]
 [0.5255148 ]
 [0.5254165 ]
 [0.52526   ]
 [0.5251516 ]
 [0.5249584 ]
 [0.52478546]
 [0.524622  ]
 [0.5245167 ]
 [0.5244075 ]
 [0.52432895]
 [0.5243305 ]
 [0.5243243 ]
 [0.5244006 ]
 [0.52444106]
 [0.5247206 ]
 [0.52487266]
 [0.52514803]
 [0.52535105]
 [0.5256903 ]
 [0.52588665]
 [0.5261558 ]
 [0.5263597 ]
 [0.5265213 ]
 [0.52663827]
 [0.5266749 ]
 [0.52667016]
 [0.5265558 ]
 [0.5264547 ]
 [0.52636975]
 [0.52629364]
 [0.52614844]
 [0.52604496]
 [0.52596056]
 [0.5259328 ]
 [0.5260472 ]
 [0.52614063]
 [0.526315  ]
 [0.52645063]
 [0.52669704]
 [0.52684057]
 [0.52700436]
 [0.52714807]
 [0.5271196 ]
 [0.52709997]
 [0.5272138 ]
 [0.5273198 ]
 [0.5272708 ]
 [0.5272891 ]
 [0.52719855]
 [0.52720886]
 [0.52702224]
 [0.5268965 ]
 [0.5263871 ]
 [0.5260041 ]
 [0.52568316]
 [0.5254291 ]
 [0.52522457]
 [0.52505124]
 [0.5247279 ]
 [0.5244725 ]
 [0.52419126]
 [0.523996

In [37]:
# Function to get final binary classification
def classify_video(cnn_lstm_model, sequences, threshold=0.5):
    predictions = cnn_lstm_model.predict(sequences)
    binary_output = (predictions > threshold).astype(int)
    return binary_output

binary_result = classify_video(cnn_lstm_model, sequences)
print(f"Video classification: {binary_result.mean() > 0.5} (1 = Anomaly, 0 = Normal)")


24/24 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step
Video classification: True (1 = Anomaly, 0 = Normal)
